In [9]:
import google.generativeai as genai
import ast
import os
import json
import PIL.Image
from selenium.webdriver.common.by import By
from constants import *

# Load environment variables
from dotenv import load_dotenv
from selenium_functions import open_browser

load_dotenv()

# Convert the GEMINI_API_KEYS string from environment variables to a list
GEMINI_API_KEYS = os.environ.get("GEMINI_API_KEYS")
KEY_LIST = ast.literal_eval(GEMINI_API_KEYS)

url = "https://dominos.com"
file = "website.html"
prompt = "I want to see the deals"

# Randomly shuffle the list of API keys
import random
random.shuffle(KEY_LIST)

# Global index to keep track of the current key
current_api_key_index = 0

In [10]:
from lxml import html


def extract_elements_by_xpath(html_string, xpath_selector):
    # Parse the HTML
    tree = html.fromstring(html_string)

    # Apply the XPath selector
    elements = tree.xpath(xpath_selector)

    # Return a list of outer HTML for each element
    return str([html.tostring(element).decode("utf-8") for element in elements] + [xpath_selector])

In [11]:
def cycle_api_key():
    global current_api_key_index
    if current_api_key_index >= len(KEY_LIST) - 1:
        current_api_key_index = 0
    else:
        current_api_key_index += 1
    return KEY_LIST[current_api_key_index]


def generate_content_with_cycling_keys(prompt, system_prompt, image=None):
    global current_api_key_index
    # Get the current API key and cycle to the next one for future requests
    api_key = cycle_api_key()

    # Configure the generative AI model with the new API key
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            max_output_tokens=8000,
            temperature=0,
        ),
        system_instruction=system_prompt,
    )

    # Generate content using the provided prompt
    if image is None:
        response = model.generate_content(prompt, request_options={"timeout": 1000})
    else:
        response = model.generate_content(
            [prompt, image], request_options={"timeout": 1000}
        )
    return response.text

In [12]:
# Load website.html into a string
with open(file, 'r') as file:
    html_string = file.read()

user_prompt = f"""
current_page: {html_string}
Output selectors for relevant elements (divs, inputs, and images) that are relevant to the user's request.x

user: {prompt}
"""

# Generate content using the prompt and the website HTML
response = generate_content_with_cycling_keys(user_prompt, system_prompt_interpret)
print(response)
if "```json" in response:
    response = response.split("```json")[1].split("```")[0]
obj = json.loads(response)

Plan:
- I need to select all of the deals on the page.
- Each deal will need the title, description, image, and button.

```json
[
  {
    "type": "xpath",
    "selector": "//div[@data-quid='mix-and-match-deal']//h2[@class='css-1dzbqx5']"
  },
  {
    "type": "xpath",
    "selector": "//div[@data-quid='mix-and-match-deal']//p[@class=' css-7ey74b']"
  },
  {
    "type": "src",
    "selector": "/static/1.88.2/images/tiles/mixAndMatchDeal/hero.webp"
  },
  {
    "type": "xpath",
    "selector": "//div[@data-quid='mix-and-match-deal']//a[@data-quid='mix-and-match-deal-button']"
  },
  {
    "type": "xpath",
    "selector": "//div[@data-quid='perfect-combo-deal']//h2[@class='css-1dzbqx5']"
  },
  {
    "type": "xpath",
    "selector": "//div[@data-quid='perfect-combo-deal']//p[@class=' css-1jj235g']"
  },
  {
    "type": "src",
    "selector": "/static/1.88.2/images/tiles/perfectComboDeal/side.webp"
  },
  {
    "type": "xpath",
    "selector": "//div[@data-quid='perfect-combo-deal']//a[@da

In [13]:
dom_elements = ""
for element in obj:
    if element['type'] == 'xpath':
        dom_elements += extract_elements_by_xpath(html_string, element["selector"])
        dom_elements += "\n"
    else:
        dom_elements += f"src: {element['selector']}\n"

In [14]:
print(dom_elements)

['<h2 class="css-1dzbqx5"><span class=" css-1uwlllm" data-quid="mix-and-match-deal-flag">Mix &amp; Match Deal</span></h2>', "//div[@data-quid='mix-and-match-deal']//h2[@class='css-1dzbqx5']"]
['<p class=" css-7ey74b" data-quid="mix-and-match-deal-headline1">Choose Any 2 or More</p>', "//div[@data-quid='mix-and-match-deal']//p[@class=' css-7ey74b']"]
src: /static/1.88.2/images/tiles/mixAndMatchDeal/hero.webp
['<a data-quid="mix-and-match-deal-button" class="css-1mi7hka" href="/en/restaurants?couponCode_0=MIXANDMATCH&amp;partnerCode=DOMINOS&amp;so=hp&amp;panelname=mixandmatch">Order Now<span class="visually-hidden" style="border: 0px; clip: rect(0px, 0px, 0px, 0px); height: 1px; margin: -1px; overflow: hidden; padding: 0px; position: absolute; white-space: nowrap; width: 1px;">&#160;Mix &amp; Match Deal</span></a>', "//div[@data-quid='mix-and-match-deal']//a[@data-quid='mix-and-match-deal-button']"]
['<h2 class="css-1dzbqx5"><span class=" css-1uwlllm" data-quid="perfect-combo-deal-flag">

In [15]:
design = PIL.Image.open("design.png")
response = generate_content_with_cycling_keys(
    design_schema
    + "\n\n"
    + dom_elements
    + "\n\n"
    + f"Only output <div>, <button>, <input>, <img>, and <select> elements. Do not use Tailwind Classes\nBase Url for images (if any): {url} \n\n",
    system_prompt_generate,
)

In [16]:
print(response)

```html
<div class='container flex flex-col gap-8 p-8 bg-[#DEDEDE]'>
    <div class='input flex flex-col gap-4'>
       <div class='heading font-inter font-bold text-2xl text-[#2B2B2B]'>Mix &amp; Match Deal</div>
       <div class='subheadings font-poppins font-semibold text-lg text-[#646464]'>Choose Any 2 or More</div>
    </div>
    <img class='img w-full' src='https://dominos.com/static/1.88.2/images/tiles/mixAndMatchDeal/hero.webp'>
    <button class='button primary-button font-poppins font-medium text-base text-white py-2 px-4' special-id='//div[@data-quid='mix-and-match-deal']//a[@data-quid='mix-and-match-deal-button']'">
        Order Now
    </button>
    <div class='input flex flex-col gap-4'>
       <div class='heading font-inter font-bold text-2xl text-[#2B2B2B]'>Perfect Combo Deal</div>
       <div class='subparagraphs font-poppins font-medium text-base text-[#646464]'>California pricing higher</div>
    </div>
    <img class='img w-full' src='https://dominos.com/static/1.8